In [ ]:
!pip3 install yfinance
!pip3 install spacy
!pip3 install bs4

In [ ]:
!pip3 install spacy

In [ ]:
!python -m spacy download en

In [342]:
import numpy as np 
import pandas as pd 
import os
import yfinance as yf
from spacy.tokens import Span
import spacy
from spacy.matcher import Matcher
from heapq import nlargest
import bs4 as bs
import requests
from spacy import displacy
from spacy_lookup import Entity
from collections import Counter
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from string import punctuation
import json

nlp = spacy.load("en")


In [297]:
data = pd.read_csv('./us_equities_news_dataset_clean.csv')

In [343]:
dictionary = []
with open("sp500.json", "r") as inputfile:
    dictionary = json.load(inputfile)
dictionary = dict(dictionary)

In [344]:
longnames = []
with open("sp500longnames.json", "r") as inputfile:
    longnames = json.load(inputfile)
longnames = dict(longnames)
tickers = []
with open("sp500tickers.json", "r") as inputfile:
    tickers = json.load(inputfile)
tickers = dict(tickers)

In [345]:
def add_event_ent(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="STOCK")
    add = True
    for i in doc.ents:
        check_start = i.start
        check_end = i.end
        if (check_start <= start and start <  check_end) or (check_start <= end and end <  check_end):
            add = False
            break
    if add:
        doc.ents += (entity,)

In [346]:
matcher = Matcher(nlp.vocab)
for i in longnames:
    pattern = [{"LOWER": i.lower()}]
    matcher.add("", [pattern], on_match=add_event_ent)
for i in tickers:
    pattern = [{"ORTH": i}]
    matcher.add("", [pattern], on_match=add_event_ent)

In [347]:
index = 0
print( 'Ticker: '+data['ticker'][index])
tokenizer = Tokenizer(nlp.vocab)
tokens = tokenizer(data['content'][index])
print(tokens[428:431])
doc = nlp(data['content'][index])
matches = matcher(doc)
displacy.render(doc, style="ent")

Ticker: INTC
Intel Corporation Price


In [348]:
def symbol_lookup(company, lookup_dictionary):
        current_name = company.strip().lower()
        current_name = current_name.replace(',', ' ')
        current_name = current_name.replace('.', ' ')
        current_name = current_name.replace('-', ' ')
        split = [j for j in current_name.split(' ')]
        full_name = ' '.join(split)
        first_name = split[0]
        if full_name in lookup_dictionary:
            return lookup_dictionary[full_name]
        elif first_name in lookup_dictionary:
            return lookup_dictionary[first_name]

In [349]:
for i in range(14300,15000):
    doc = nlp(data['title'][i]+'\n'+data['content'][i])
    matches = matcher(doc)
    count = {}
    countSPY = {}
    for j in doc.ents:
        if j.label_ == 'ORG' or j.label_ == 'PERSON' or j.label_ == 'STOCK' or j.label_ == 'PRODUCT':
            symbol = symbol_lookup(j.text, dictionary)
            if symbol:
                if symbol not in countSPY:
                    countSPY[symbol] = 1
                else:
                    countSPY[symbol] = 1 + countSPY[symbol]        
    if data['ticker'][i] not in countSPY:
        print(i)
        print(data['ticker'][i])
        print(countSPY)
        displacy.render(doc, style="ent")
        break

14542
CMG
{'MSCI': 2, 'O': 1, 'AXP': 2, 'ADM': 1, 'DG': 1, 'GPS': 1, 'IBM': 2, 'WMT': 1}


In [ ]:
correct = 0
data = data.sample(frac = 1)
for i in range(10000):
    if i % 100 == 0 and i > 0:
        print(correct / i * 100)
    doc = nlp(data['title'][i]+'\n'+data['content'][i])
    matches = matcher(doc)
    count = {}
    countSPY = {}
    for j in doc.ents:
        if j.label_ == 'ORG' or j.label_ == 'PERSON' or j.label_ == 'STOCK' or j.label_ == 'PRODUCT':
            symbol = symbol_lookup(j.text, dictionary)
            if symbol:
                if symbol not in countSPY:
                    countSPY[symbol] = 1
                else:
                    countSPY[symbol] = 1 + countSPY[symbol]      
    if data['ticker'][i] in countSPY:
        correct += 1

96.0
96.0
